In [25]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

df2016 = pd.read_csv('hockey_team_2016.csv')
df2017 = pd.read_csv('hockey_team_2017.csv')
df2018 = pd.read_csv('hockey_team_2018.csv')
df2019 = pd.read_csv('hockey_team_2019.csv')

df2016['Year'] = 2016
df2017['Year'] = 2017
df2018['Year'] = 2018
df2019['Year'] = 2019

In [26]:
df = pd.concat([df2016,df2017,df2018,df2019], ignore_index=True)


def _playoff_return(data):
    if('*' in data):
        return 1
    else:
        return 0

df["Playoff"] = df["Name"].apply(_playoff_return)

In [27]:
df.head()

,Rk,Name,AvAge,GP,W,L,OL,PTS,PTS%,GF,...,SHA,PIM/G,oPIM/G,S,S%,SA,SV%,SO,Year,Playoff
0,1,Washington Capitals*,27.9,82,56,18,8,120,0.732,248,...,5,9.2,8.9,2510,9.9,2332,0.918,3,2016,1
1,2,Dallas Stars*,28.9,82,50,23,9,109,0.665,265,...,15,8.8,9.0,2624,10.1,2371,0.904,5,2016,1
2,3,St. Louis Blues*,28.0,82,49,24,9,107,0.652,219,...,5,10.5,9.2,2474,8.9,2439,0.919,10,2016,1
3,4,Pittsburgh Penguins*,29.0,82,48,26,8,104,0.634,241,...,5,8.5,8.9,2722,8.9,2436,0.918,6,2016,1
4,5,Anaheim Ducks*,27.4,82,46,25,11,103,0.628,215,...,3,12.5,11.1,2483,8.7,2258,0.917,8,2016,1


In [28]:
from sklearn.tree import DecisionTreeClassifier
from sklearn import preprocessing
from sklearn.tree import export_graphviz
import pydotplus

data_encoded = df.apply(preprocessing.LabelEncoder().fit_transform)
print(data_encoded)

clf = DecisionTreeClassifier(criterion='entropy', max_depth=3)

clf.fit(data_encoded[['PTS', 'W', 'L', 'SRS']], data_encoded['Playoff'])

dot_data = export_graphviz(clf, out_file=None, feature_names=['PTS', 'W', 'L', 'SRS'])

graph = pydotplus.graph_from_dot_data(dot_data)
graph.write_png('hockey_tree.png')

     Rk  Name  AvAge  GP   W   L  OL  PTS  PTS%  GF   ...     SHA  PIM/G  \
0     0    51     17   0  30   1   4   49    49  49   ...       4     27   
1     1    17     27   0  24   5   5   42    42  60   ...      14     23   
2     2    44     18   0  23   6   5   40    40  22   ...       4     39   
3     3    41     28   0  22   8   4   37    37  42   ...       4     20   
4     4     1     12   0  20   7   7   36    36  19   ...       2     52   
5     5    11     24   0  21   8   5   36    36  36   ...       2     10   
6     6    23     30   0  21   8   5   36    36  34   ...       4     38   
7     7    25     21   0  22  10   2   35    35  26   ...       2     40   
8     8    36     34   0  20   9   5   34    34  35   ...       4     21   
9     9    34     17   0  19   9   6   33    33  29   ...       5     23   
10   10    42     29   0  20  12   2   31    31  38   ...       6     24   
11   11    46     11   0  20  13   1   30    30  27   ...       6     33   
12   12    4

True

In [29]:
#X = df.drop(columns = ['Name', 'Playoff', 'Year', 'Rk', 'AvAge', 'GP', 'OL', 'PTS%', 'GF'])
X = df[['PTS', 'W', 'L', 'SRS']]
y = df.Playoff

X_train, X_test, y_train, y_test = train_test_split(X, y)

In [38]:
#dtc = DecisionTreeClassifier()
#dtc.fit(X_train, y_train)
clf.score(X_test, y_test)

0.5806451612903226

In [40]:
clf.predict([[30, 50, 49, .75]], check_input=True)

array([1])

In [44]:
def predict_playoff(dt, stats_array):
    playoffs = dt.predict(stats_array, check_input=True)
    if playoffs == 1:
        print('Going to playoffs!')
    else:
        print('Daaaaamn')
        
stats = [[10, 1, 49, .06]]
        
predict_playoff(clf, stats)

Daaaaamn
